## Read Data

In [2]:
# Load data from a CSV
file_location = "/FileStore/tables/game_skater_stats.csv"
df = spark.read.format("CSV").option("inferSchema", True).option("header", True).load(file_location)
display(df.take(5))

game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
2012030221,8471958,3,1925,0,0,0,3,0,0,0,0,0,1,0,0,0,5,0,1406,342,177
2012030221,8471339,3,1597,1,0,2,3,0,0,0,0,0,1,3,0,0,2,1,1099,315,183
2012030221,8471873,3,1695,0,0,1,2,0,0,0,0,0,0,0,0,0,2,0,1488,99,108
2012030221,8473432,3,957,0,0,3,5,0,0,2,0,0,0,1,0,0,0,0,957,0,0
2012030221,8470192,3,859,0,0,1,0,0,0,2,0,0,1,0,0,0,2,0,859,0,0


## Write Data

In [4]:
# Save as CSV and parquet

# DBFS
df.write.save('/FileStore/parquet/game__stats', format='parquet')

# S3
#df.write.parquet("s3a://my_bucket/game_skater_stats", mode="overwrite")

# DBFS
df.write.save('/FileStore/parquet/game__stats.csv', format='csv')

# S3
#df.coalesce(1).write.format("com.databricks.spark.csv")
#   .option("header", "true").save("s3a://my_bucket/game_skater_stats.csv")


## Transforming Data

In [6]:
df.createOrReplaceTempView("stats")

display(spark.sql("""
  select player_id, sum(1) as games, sum(goals) as goals
  from stats
  group by 1
  order by 3 desc
  limit 5
"""))

player_id,games,goals
8471214,520,299
8471675,522,221
8474141,499,216
8470794,515,207
8475765,465,200


In [7]:
# player names
file_location = "/FileStore/tables/player_info.csv"
names = spark.read.format("CSV").option("inferSchema", True).option("header", True).load(file_location)
#display(names)

In [8]:
df.createOrReplaceTempView("stats")

top_players = spark.sql("""
select player_id, sum(1) as games, sum(goals) as goals
from stats
group by 1
order by 3 desc
limit 5
""")

top_players.createOrReplaceTempView("top_players")
names.createOrReplaceTempView("names")

display(spark.sql("""
select p.player_id, goals, firstName, lastName
from top_players p
join names n
  on p.player_id = n.player_id
order by 2 desc  
"""))


player_id,goals,firstName,lastName
8471214,299,Alex,Ovechkin
8471675,221,Sidney,Crosby
8474141,216,Patrick,Kane
8470794,207,Joe,Pavelski
8475765,200,Vladimir,Tarasenko


In [9]:
display(spark.sql("""
select cast(substring(game_id, 1, 4) || '-' 
  || substring(game_id, 5, 2) || '-01' as Date) as month
  , sum(goals)/count(distinct game_id) as goals_per_goal
from stats
group by 1
order by 1
"""))

month,goals_per_goal
2012-02-01,5.3069444444444445
2012-03-01,5.023255813953488
2013-02-01,5.342276422764227
2013-03-01,5.591397849462366
2014-02-01,5.324390243902439
2014-03-01,5.033707865168539
2015-02-01,5.337398373983739
2015-03-01,5.252747252747253
2016-02-01,5.4504065040650405
2016-03-01,5.160919540229885


In [10]:

display(spark.sql("""
select cast(goals/shots * 50 as int)/50.0 as Goals_per_shot, sum(1) as Players 
from (
  select player_id, sum(shots) as shots, sum(goals) as goals
  from stats
  group by 1
  having goals >= 5
)  
group by 1
order by 1
"""))  
  

Goals_per_shot,Players
0.0,1
0.02,83
0.04,157
0.06,154
0.08,169
0.1,171
0.12,117
0.14,37
0.16,13
0.18,4


## MLlib: Linear Regression

In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=['shots', 'hits', 'assists', 'penaltyMinutes', 'timeOnIce', 'takeaways'], outputCol="features" )
train_df = assembler.transform(df) 

lr = LinearRegression(featuresCol = 'features', labelCol='goals')
lr_model = lr.fit(train_df)

trainingSummary = lr_model.summary
print("Coefficients: " + str(lr_model.coefficients))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("R2: %f" % trainingSummary.r2)

Coefficients: [0.092664303446,-0.00633910298663,0.00602854937487,-0.000485772787063,-4.61822622641e-05,0.0176748286452]
RMSE: 0.375613
R2: 0.124695

## Pandas UDFs

In [14]:
# creating a linear fit for a single player

df.createOrReplaceTempView("stats")

sample_pd = spark.sql("""
select * from stats
where player_id = 8471214
""").toPandas()

from scipy.optimize import leastsq
import numpy as np

def fit(params, x, y):
    return (y - (params[0] + x * params[1] ))  

result = leastsq(fit, [1, 0], args=(sample_pd.shots, sample_pd.hits))
print(result)


(array([ 2.87826126, -0.01760805]), 1)

In [15]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
import pandas as pd

schema = StructType([StructField('ID', LongType(), True),
                     StructField('p0', DoubleType(), True),
                     StructField('p1', DoubleType(), True)])  

  
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def analyze_player(sample_pd):
  
    if (len(sample_pd.shots) <= 1):
        return pd.DataFrame({'ID': [sample_pd.player_id[0]], 'p0': [ 0 ], 'p1': [ 0 ]})
    
    result = leastsq(fit, [1, 0], args=(sample_pd.shots, sample_pd.hits))
    return pd.DataFrame({'ID': [sample_pd.player_id[0]], 'p0': [result[0][0]], 'p1': [result[0][1]]})

player_df = df.groupby('player_id').apply(analyze_player)
display(player_df.take(5))

ID,p0,p1
8470085,2.344963791971333,-0.15734035549738007
8471859,0.6199999999991705,-0.036190476190554856
8475765,0.6632097778743309,-0.0035926360505775527
8476426,-2.1813661987835076E-12,1.6666666666703023
8476439,2.185808176453509,0.08120753035553108
